In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
df= pd.read_excel('02 Ch Genes ordenados con vecinos.xlsx',engine='openpyxl',sheet_name=None,skiprows=[0],usecols=range(18))

In [3]:
nombres=list(df.keys())
nombres

['Información',
 'C1 Milagros',
 'C2 Wilmer',
 'C3 Gisell',
 'C4 Jerica',
 'C5 Manuel',
 'C6 Jerica',
 'C7 Mariana',
 'C8 Rosha',
 'C9 Jerica',
 'C10 Lucero',
 'C11 Nicole',
 'C12 Gisell',
 'C13 Lucero',
 'C14 Wilmer',
 'C15Milagros',
 'C16',
 'C17',
 'C18',
 'Hoja 20',
 'Hoja 21',
 'Hoja 22',
 'Hoja 23']

In [23]:
hojas=[]
for nombre in nombres: 
    if 'Gen Abrev' in list(df.get(nombre).columns):
        hojas.append(df.get(nombre)[df.get(nombre)['Gen Abrev'].notnull()])

In [24]:
genes=[]
for hoja in hojas: 
    genes.append(hoja['Gen Abrev'].tolist())

In [27]:
genes[16]

['SFI1',
 'LOC102173595',
 'EIF4ENIF1',
 'DRG1',
 'PATZ1',
 'PIK3IP1',
 'LIMK2',
 'LOC108637845',
 'CRKL',
 'LOC108637844',
 'RNF185',
 'PLA2G3',
 'INPP5J',
 'LOC102171091',
 'SMTN',
 'LOC106503045',
 'LOC102169320',
 'MORC2',
 'OSBP2',
 'C17H5orf52',
 'DUSP18',
 'SLC35E4',
 'TCN2',
 'PES1',
 'GAL3ST1',
 'LOC102190250',
 'LOC102169408',
 'SEC14L3',
 'MTFP1',
 'SEC14L2',
 'RNF215',
 'CCDC157',
 'SF3A1',
 'TBC1D10A',
 'GATSL3',
 'OSM',
 'LIF',
 'TRNAG-CCC',
 'HORMAD2',
 'MTMR3',
 'ASCC2',
 'TRNAE-CUC',
 'LOC102187154',
 'ZMAT5',
 'CABP7',
 'NF2',
 'NIPSNAP1',
 'THOC5',
 'NEFH',
 'LOC106503044',
 'AP1B1',
 'RASL10A',
 'GAS2L1',
 'EWSR1',
 'RHBDD3',
 'EMID1',
 'LOC102189423',
 'KREMEN1',
 'C17H22orf31',
 'ZNRF3',
 'XBP1',
 'CCDC117',
 'CMKLR1',
 'FICD',
 'SART3',
 'ISCU',
 'TMEM119',
 'SELPLG',
 'CORO1C',
 'LOC106503038',
 'SSH1',
 'DAO',
 'SVOP',
 'TRNAG-UCC',
 'USP30',
 'ALKBH2',
 'UNG',
 'ACACB',
 'FOXN4',
 'MYO1H',
 'KCTD10',
 'UBE3B',
 'MMAB',
 'MVK',
 'FAM222A',
 'TRPV4',
 'GLTP',
 '

In [28]:
links=[]
for lista in genes:
    links1=[]
    for gen in lista: 
        if type(gen)==str:
            url = "https://www.uniprot.org/uniprot/?query="+gen+"+Capra+Hircus&sort=score"
            #print(url)
            links1.append(url)
    links.append(links1)

In [40]:
# Se crea una función para determinar el número de genes que se desea extraer del excel
def num_genes():
    #se pregunta si se desea un número exacto o todos los genes
    x=input("¿Desea extraer un número exacto de genes?, S/N")
    #si la respuesta es si
    if x=='S':
        #se ingresa la cantidad de datos que se solicita
        Ngenes=int(input("Ingrese el número de genes que desea extraer de cada hoja: "))
        return Ngenes
    #si la respuesta es no
    elif x=='N':
        # se retorna un número muy grande que sobrepase la cantidad de genes en las hojas 
        return 99999999
    # en caso de colocar una respuesta erronea se vuelve a la pregunta inicial
    else:
        return num_genes()

In [41]:
from tqdm.notebook import tqdm
writer = pd.ExcelWriter('GenesRepository.xlsx', engine='openpyxl')
#Se llama a la función en una variable 
n_genes=num_genes()
print("Progreso de las hojas del excel")
for j in tqdm(range(len(links))):#se va de hoja en hoja 
    lista = links[j]
    htmlList=[]
    datos=[]
    print("\nProgreso de los genes en la hoja actual")
    #Se solicita el mínimo de entre la cantidad de elementos de la lista y la cantidad de genes solicitados 
    for i in tqdm(range(min(len(lista),n_genes))):#Se va de gen en gen 
        print("\rHoja Actual: "+nombres[j+1]+"     Gen actual: "+genes[j][i],end="")
        link=lista[i]
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'lxml')
        html=soup
        datos1=[]
        if html.find("td",{"class":"entryID"})!=None:
            datos1.append(html.find("td",{'class':"entryID"}).a.text)
            datos1.append(html.find("td",{'class':"entryID"}).find_next_sibling("td").text)
            link='https://www.uniprot.org'+html.find("td",{'class':"entryID"}).a['href']
            r1=requests.get(link)
            soup1=BeautifulSoup(r1.text, 'lxml')
            datos1.append(soup1.find("div",{'id':"content-protein"}).h1.text)
            if soup1.find("div",{'id':"content-gene"}).h2!=None:
                datos1.append(soup1.find("div",{'id':"content-gene"}).h2.text)
            else:
                datos1.append(None)
            datos1.append(soup1.find("div",{'id':"content-organism"}).text.split(" (")[0])
            datos1.append(soup1.find("div",{'id':"content-status"}).a.text)
            tag = soup1.find(lambda tag:tag.name=="h4" and "GO - Biological process" in tag.text)
            if(tag==None):
                datos1.append(None)
            else: 
                lists= tag.next_sibling
                listaTemp=[]
                for l in lists:
                    listaTemp.append(l.text.split(" Source")[0])
                datos1.append(",".join(listaTemp))
        datos.append(datos1)
    df_datos=pd.DataFrame(datos, columns=["Entry","Entry name","Protein","Gene","Organism","Status","GO - Biological process"])
    df_genes=pd.DataFrame(genes[j],columns=["Gen Abrev"])
    df_final=pd.concat([df_genes.head(20),df_datos],axis=1)
    df_final.index.name='N°'
    df_final=df_final.fillna(value='NaN')
    df_final.to_excel(writer,sheet_name=nombres[j+1])
    writer.save()

¿Desea extraer un número exacto de genes?, S/Nm
¿Desea extraer un número exacto de genes?, S/Ngf
¿Desea extraer un número exacto de genes?, S/Nf
¿Desea extraer un número exacto de genes?, S/Nhgfgh
¿Desea extraer un número exacto de genes?, S/NS
Ingrese el número de genes que desea extraer de cada hoja: 15
Progreso de las hojas del excel


  0%|          | 0/18 [00:00<?, ?it/s]


Progreso de los genes en la hoja actual


  0%|          | 0/15 [00:00<?, ?it/s]

Hoja Actual: C1 Milagros     Gen actual: SLC12A890548
Progreso de los genes en la hoja actual


...

Hoja Actual: C16     Gen actual: LOC102190981
Progreso de los genes en la hoja actual


  0%|          | 0/15 [00:00<?, ?it/s]

Hoja Actual: C17     Gen actual: SMTN02171091
Progreso de los genes en la hoja actual


  0%|          | 0/15 [00:00<?, ?it/s]

Hoja Actual: C18     Gen actual: LOC102184183